# 1. 針對 churn 欄位使用 Stratified sampling 從原本的資料集中取 60%的資料

## 1.1 載入 customer_churn.csv 為 pandas DataFrame 格式

In [1]:
import pandas as pd
df = pd.read_csv("customer_churn.csv")
df

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,160
1,50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,4,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,121
2,50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,4,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120
3,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,8,0,23.0,0.0,1.0,3.0,134
4,50005,1,0.0,Phone,1,12.0,CC,Male,NaN,3,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5635,55626,0,10.0,Computer,1,30.0,Credit Card,Male,3.0,2,Laptop & Accessory,1,Married,6,0,18.0,1.0,2.0,4.0,151
5636,55627,0,13.0,Mobile Phone,1,13.0,Credit Card,Male,3.0,5,Fashion,5,Married,6,0,16.0,1.0,2.0,NaN,225
5637,55628,0,1.0,Mobile Phone,1,11.0,Debit Card,Male,3.0,2,Laptop & Accessory,4,Married,3,1,21.0,1.0,2.0,4.0,186
5638,55629,0,23.0,Computer,3,9.0,Credit Card,Male,4.0,5,Laptop & Accessory,4,Married,4,0,15.0,2.0,2.0,9.0,179


## 1.2.列出資料筆數、屬性數量、每個欄位的空值個數以及各類別(target)的資料筆數

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5640 entries, 0 to 5639
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CustomerID                   5640 non-null   int64  
 1   Churn                        5640 non-null   int64  
 2   Tenure                       5376 non-null   float64
 3   PreferredLoginDevice         5640 non-null   object 
 4   CityTier                     5640 non-null   int64  
 5   WarehouseToHome              5389 non-null   float64
 6   PreferredPaymentMode         5640 non-null   object 
 7   Gender                       5640 non-null   object 
 8   HourSpendOnApp               5385 non-null   float64
 9   NumberOfDeviceRegistered     5640 non-null   int64  
 10  PreferedOrderCat             5640 non-null   object 
 11  SatisfactionScore            5640 non-null   int64  
 12  MaritalStatus                5640 non-null   object 
 13  NumberOfAddress   

## 1.3 資料集中有部分資料重覆出現，請刪除重覆多餘的資料 (僅保留一筆)，並列出剩餘的資料筆數。

In [3]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5630 entries, 0 to 5639
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CustomerID                   5630 non-null   int64  
 1   Churn                        5630 non-null   int64  
 2   Tenure                       5366 non-null   float64
 3   PreferredLoginDevice         5630 non-null   object 
 4   CityTier                     5630 non-null   int64  
 5   WarehouseToHome              5379 non-null   float64
 6   PreferredPaymentMode         5630 non-null   object 
 7   Gender                       5630 non-null   object 
 8   HourSpendOnApp               5375 non-null   float64
 9   NumberOfDeviceRegistered     5630 non-null   int64  
 10  PreferedOrderCat             5630 non-null   object 
 11  SatisfactionScore            5630 non-null   int64  
 12  MaritalStatus                5630 non-null   object 
 13  NumberOfAddress   

## 1.4 填補空值及其他資料前處理 (e.g. feature encoding)

In [4]:
## 刪掉 CustomerID
df = df.drop(['CustomerID'],axis=1)
df

,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,160
1,1,NaN,Phone,1,8.0,UPI,Male,3.0,4,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,121
2,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,4,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120
3,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,8,0,23.0,0.0,1.0,3.0,134
4,1,0.0,Phone,1,12.0,CC,Male,NaN,3,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5635,0,10.0,Computer,1,30.0,Credit Card,Male,3.0,2,Laptop & Accessory,1,Married,6,0,18.0,1.0,2.0,4.0,151
5636,0,13.0,Mobile Phone,1,13.0,Credit Card,Male,3.0,5,Fashion,5,Married,6,0,16.0,1.0,2.0,NaN,225
5637,0,1.0,Mobile Phone,1,11.0,Debit Card,Male,3.0,2,Laptop & Accessory,4,Married,3,1,21.0,1.0,2.0,4.0,186
5638,0,23.0,Computer,3,9.0,Credit Card,Male,4.0,5,Laptop & Accessory,4,Married,4,0,15.0,2.0,2.0,9.0,179


In [5]:
# 確認欄位空值
df.isna().sum()

Churn                            0
Tenure                         264
PreferredLoginDevice             0
CityTier                         0
WarehouseToHome                251
PreferredPaymentMode             0
Gender                           0
HourSpendOnApp                 255
NumberOfDeviceRegistered         0
PreferedOrderCat                 0
SatisfactionScore                0
MaritalStatus                    0
NumberOfAddress                  0
Complain                         0
OrderAmountHikeFromlastYear    265
CouponUsed                     256
OrderCount                     258
DaySinceLastOrder              307
CashbackAmount                   0
dtype: int64

In [6]:
# 填補欄位空值 numeric 用 mean 回填 nominal 用 mode 回填
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].mean())
df['WarehouseToHome'] = df['WarehouseToHome'].fillna(df['WarehouseToHome'].mean())
df['HourSpendOnApp'] = df['HourSpendOnApp'].fillna(df['HourSpendOnApp'].mean())
df['OrderAmountHikeFromlastYear'] = df['OrderAmountHikeFromlastYear'].fillna(df['OrderAmountHikeFromlastYear'].mean())
df['DaySinceLastOrder'] = df['DaySinceLastOrder'].fillna(df['DaySinceLastOrder'].mean())
df['CouponUsed'] = df['CouponUsed'].fillna(df['CouponUsed'].mean())
df['OrderCount'] = df['OrderCount'].fillna(df['OrderCount'].mean())

In [7]:
# 檢查是否有遺漏未填補的缺失值
df.isna().sum()


Churn                          0
Tenure                         0
PreferredLoginDevice           0
CityTier                       0
WarehouseToHome                0
PreferredPaymentMode           0
Gender                         0
HourSpendOnApp                 0
NumberOfDeviceRegistered       0
PreferedOrderCat               0
SatisfactionScore              0
MaritalStatus                  0
NumberOfAddress                0
Complain                       0
OrderAmountHikeFromlastYear    0
CouponUsed                     0
OrderCount                     0
DaySinceLastOrder              0
CashbackAmount                 0
dtype: int64

In [8]:
## 將 Churn 欄位換成 nominal 的資料型態
df['Churn'] = df['Churn'].astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5630 entries, 0 to 5639
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Churn                        5630 non-null   string 
 1   Tenure                       5630 non-null   float64
 2   PreferredLoginDevice         5630 non-null   object 
 3   CityTier                     5630 non-null   int64  
 4   WarehouseToHome              5630 non-null   float64
 5   PreferredPaymentMode         5630 non-null   object 
 6   Gender                       5630 non-null   object 
 7   HourSpendOnApp               5630 non-null   float64
 8   NumberOfDeviceRegistered     5630 non-null   int64  
 9   PreferedOrderCat             5630 non-null   object 
 10  SatisfactionScore            5630 non-null   int64  
 11  MaritalStatus                5630 non-null   object 
 12  NumberOfAddress              5630 non-null   int64  
 13  Complain          

In [9]:
#將除了target (Churn) 外的非 numeric 欄位轉換成 numeric 欄位
dummyColumn = pd.get_dummies(df[['PreferredLoginDevice','PreferredPaymentMode','Gender','PreferedOrderCat','MaritalStatus']])
df_dummy_column = pd.DataFrame(dummyColumn)
df = pd.concat([df, df_dummy_column],axis=1)
# drop 掉不是 numeric 的欄位
df = df.drop(['PreferredLoginDevice','PreferredPaymentMode','Gender','PreferedOrderCat','MaritalStatus'], axis=1)
df.info()                    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5630 entries, 0 to 5639
Data columns (total 35 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Churn                                  5630 non-null   string 
 1   Tenure                                 5630 non-null   float64
 2   CityTier                               5630 non-null   int64  
 3   WarehouseToHome                        5630 non-null   float64
 4   HourSpendOnApp                         5630 non-null   float64
 5   NumberOfDeviceRegistered               5630 non-null   int64  
 6   SatisfactionScore                      5630 non-null   int64  
 7   NumberOfAddress                        5630 non-null   int64  
 8   Complain                               5630 non-null   int64  
 9   OrderAmountHikeFromlastYear            5630 non-null   float64
 10  CouponUsed                             5630 non-null   float64
 11  Orde

In [10]:
# 資料標準化
from sklearn.preprocessing import StandardScaler
df_scaled = df.drop('Churn',axis=1)
scaler = StandardScaler().fit(df_scaled)
df_scaled = scaler.transform(df_scaled)
df_scaled = pd.DataFrame(df_scaled)
# reset Churn
df_churn = pd.DataFrame(df['Churn'])
df_churn = df_churn.reset_index().drop("index",axis=1)
df_scaled = pd.concat([df_churn,df_scaled],axis=1)
df_scaled.columns = df.columns
df_scaled

,Churn,Tenure,CityTier,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,...,Gender_Male,PreferedOrderCat_Fashion,PreferedOrderCat_Grocery,PreferedOrderCat_Laptop & Accessory,PreferedOrderCat_Mobile,PreferedOrderCat_Mobile Phone,PreferedOrderCat_Others,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,1,-0.741002,1.469771,-1.156091,0.097069,-0.672900,-0.772992,1.852616,1.584290,-1.312273,...,-1.227468,-0.414657,-0.280257,1.321492,-0.409643,-0.539982,-0.221808,-0.421108,-1.062709,1.461076
1,1,0.000000,-0.715286,-0.916235,0.097069,0.303750,-0.048392,1.078430,1.584290,-0.197324,...,0.814685,-0.414657,-0.280257,-0.756720,2.441149,-0.539982,-0.221808,-0.421108,-1.062709,1.461076
2,1,0.000000,-0.715286,1.722175,-1.320723,0.303750,-0.048392,0.691336,1.584290,-0.476062,...,0.814685,-0.414657,-0.280257,-0.756720,2.441149,-0.539982,-0.221808,-0.421108,-1.062709,1.461076
3,1,-1.219847,1.469771,-0.076741,-1.320723,0.303750,1.400807,1.465523,-0.631198,2.032574,...,0.814685,-0.414657,-0.280257,1.321492,-0.409643,-0.539982,-0.221808,-0.421108,-1.062709,1.461076
4,1,-1.219847,-0.715286,-0.436525,0.000000,-0.672900,1.400807,-0.469944,-0.631198,-1.312273,...,0.814685,-0.414657,-0.280257,-0.756720,2.441149,-0.539982,-0.221808,-0.421108,-1.062709,1.461076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,0,-0.022733,-0.715286,1.722175,0.097069,-1.649551,-1.497592,0.691336,-0.631198,0.638888,...,0.814685,-0.414657,-0.280257,1.321492,-0.409643,-0.539982,-0.221808,-0.421108,0.940992,-0.684427
5626,0,0.336401,-0.715286,-0.316597,0.097069,1.280401,1.400807,0.691336,-0.631198,0.081413,...,0.814685,2.411634,-0.280257,-0.756720,-0.409643,-0.539982,-0.221808,-0.421108,0.940992,-0.684427
5627,0,-1.100136,-0.715286,-0.556452,0.097069,-1.649551,0.676207,-0.469944,1.584290,1.475100,...,0.814685,-0.414657,-0.280257,1.321492,-0.409643,-0.539982,-0.221808,-0.421108,0.940992,-0.684427
5628,0,1.533515,1.469771,-0.796308,1.514862,1.280401,0.676207,-0.082850,-0.631198,-0.197324,...,0.814685,-0.414657,-0.280257,1.321492,-0.409643,-0.539982,-0.221808,-0.421108,0.940992,-0.684427


In [11]:
df_scaled.groupby("Churn").size()

Churn
0    4682
1     948
dtype: int64

## 1.5 從原本的資料集中取 60%的資料 (Stratified sampling)

In [12]:
df_stratified = df_scaled.groupby('Churn', group_keys=False).apply(lambda x: x.sample(frac=0.6))

# 2. 列出取樣後各類別的資料數量

In [13]:
df_stratified.groupby("Churn").size()

Churn
0    2809
1     569
dtype: int64

# 3. 資料前處理，並以 10 folds cross-validation 建立 Logistic Regression 及 SVM模型

### 資料前處理已經在 **1.4 填補空值及其他資料前處理 (e.g. feature encoding)** 完成，這邊就不用再做資料前處理

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
X = df_stratified.drop(['Churn'], axis=1)
y = df_stratified['Churn']

#定義交叉驗證方法
cv = KFold(n_splits=10,shuffle=True)

# 建立 LogisticRegression Model
model1 = LogisticRegression()
# 建立 SVM Model
model2 = SVC(kernel='sigmoid', C=1)

#使用 k-fold 交叉驗證評估 model
scores1 = cross_val_score(model1, X, y, scoring='accuracy',cv=cv)
scores2 = cross_val_score(model2, X, y, scoring='accuracy',cv=cv)

# 4. 針對測試資料印出兩個模型的平均 Accuracy

In [15]:
accuracyTable = pd.DataFrame({"Logistic Regression":scores1.mean(),"SVM":scores2.mean()},index=['Mean Accuracy'])
accuracyTable

,Logistic Regression,SVM
Mean Accuracy,0.897276,0.863226


# 5. 重複 1~4 題 30 次，並印出兩種模型最終的平均 Accuracy

In [16]:
scores1_list = []
scores2_list = []
for i in range(30):
    df_stratified = df_scaled.groupby('Churn', group_keys=False).apply(lambda x: x.sample(frac=0.6))
    X = df_stratified.drop(['Churn'], axis=1)
    y = df_stratified['Churn']

    #定義交叉驗證方法
    cv = KFold(n_splits=10,shuffle=True)

    # 建立 LogisticRegression Model
    model1 = LogisticRegression()
    # 建立 SVM Model
    model2 = SVC(kernel='sigmoid', C=1)

    #使用 k-fold 交叉驗證評估 model
    scores1 = cross_val_score(model1, X, y, scoring='accuracy',cv=cv)
    scores2 = cross_val_score(model2, X, y, scoring='accuracy',cv=cv)
    scores1_list = scores1_list + list(scores1)
    scores2_list = scores2_list + list(scores2)

In [17]:
accuracyTable_final = pd.DataFrame({"Logistic Regression":scores1_list,"SVM":scores2_list})
print("Accuracy: 30 times 10 folds cross-validation")
accuracyTable_final

Accuracy: 30 times 10 folds cross-validation


,Logistic Regression,SVM
0,0.905325,0.875740
1,0.887574,0.843195
2,0.890533,0.840237
3,0.890533,0.843195
4,0.890533,0.875740
...,...,...
295,0.905325,0.857988
296,0.893491,0.819527
297,0.896450,0.849112
298,0.905045,0.845697


## 5.1 兩種模型的最終平均 Accuracy

In [18]:
import numpy as np
ndarray1 = np.array(scores1_list)
ndarray2 = np.array(scores2_list)
print("Mean Accuracy: 30 times 10 folds cross-validation")
accuracyTable_final_mean = pd.DataFrame({"Logistic Regression":ndarray1.mean(),"SVM":ndarray2.mean()},index=['Mean Accuracy'])
accuracyTable_final_mean

Mean Accuracy: 30 times 10 folds cross-validation


,Logistic Regression,SVM
Mean Accuracy,0.890496,0.851182


# 6. 根據模型於 30 次 10 folds cross-validation 的 Accuracy，以 paired t-test 比較兩種 模型，並說明結論

![](https://i.imgur.com/QPX7tyx.jpg)

In [19]:
import scipy.stats as stats
# Performing the paired sample t-test
stats.ttest_ind(scores1_list, scores2_list,equal_var=False)

Ttest_indResult(statistic=26.241844333761424, pvalue=3.246661134499933e-101)

做完 paired t-test 後發現，pvalue遠小於0.05，可以拒絕虛無假設，表示 Logistic Regression 和 SVM 的 30 次 10 folds cross-validation 的平均 Accuracy 有顯著的差異。
而從平均 Accuracy 來看，使用 Logistic Regression 是比較好的選擇。